In [3]:
import tensorflow as tf
import string
import requests
import pandas as pd


In [4]:
response = requests.get('https://github.com/Manikandan029/TNSDC-Generative-AI/blob/main/adele.txt')


In [ ]:
data=response.text.splitlines()
print(data[:10])

In [10]:
len(data)

2400

In [11]:
len(" ".join(data))

91330

In [12]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout, Conv1D, MaxPooling1D
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [13]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(data)

In [14]:
encoded_text=tokenizer.texts_to_sequences(data)
encoded_text[:10]

[[254, 21, 219, 725],
 [117, 8, 80, 153, 3, 133],
 [14, 10, 726, 727],
 [41, 56, 2, 603, 3, 728, 1, 68, 517, 2, 40, 3, 518, 41],
 [1, 23, 107, 189, 300, 9, 57],
 [286, 35, 46, 10, 230],
 [2, 83, 134, 4, 519, 8, 120],
 [1, 37, 520, 102, 19, 27, 25, 254, 21, 328, 11],
 [27, 209, 11, 13, 9, 124],
 [42, 67, 210, 125]]

In [15]:
wc=tokenizer.word_counts

In [16]:
wi=tokenizer.word_index

In [17]:
print(f"Number of unique words and total vocab size: {len(tokenizer.word_counts)+1}")
vocab_size=len(tokenizer.word_counts)+1

Number of unique words and total vocab size: 1396


In [18]:
x=["play this song"]

In [19]:
tokenizer.texts_to_sequences(x)


[[241, 44, 409]]

In [20]:
 encoded_text[:10]

[[254, 21, 219, 725],
 [117, 8, 80, 153, 3, 133],
 [14, 10, 726, 727],
 [41, 56, 2, 603, 3, 728, 1, 68, 517, 2, 40, 3, 518, 41],
 [1, 23, 107, 189, 300, 9, 57],
 [286, 35, 46, 10, 230],
 [2, 83, 134, 4, 519, 8, 120],
 [1, 37, 520, 102, 19, 27, 25, 254, 21, 328, 11],
 [27, 209, 11, 13, 9, 124],
 [42, 67, 210, 125]]

In [21]:
data_list=[]
for i in encoded_text:
    if len(i)>1:
        for j in range(2,len(i)):
            data_list.append(i[:j])

In [22]:
max_length=20

In [23]:
sequences=pad_sequences(data_list,maxlen=max_length,padding="pre") # we set the lenght size equal to 20
sequences[:5]

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0, 254,  21],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0, 254,  21, 219],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0, 117,   8],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0, 117,   8,  80],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0, 117,   8,  80, 153]], dtype=int32)

In [24]:
sequences.shape

(14231, 20)

In [25]:

X=sequences[:,:-1]
y=sequences[:,-1]
print("X values")
print(X[:5]) # 5 sample
print("-"*30)
print("X values")
print(y[:5])

X values
[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
  254]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 254
   21]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
  117]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 117
    8]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 117   8
   80]]
------------------------------
X values
[ 21 219   8  80 153]


In [26]:
X.shape,y.shape


((14231, 19), (14231,))

In [27]:
y=to_categorical(y,num_classes=vocab_size)
#since unique word number is vocab_size, thus there is vocab_size classes
print(y[:5]) # 5 sample
print("Shape of y",y.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Shape of y (14231, 1396)


In [28]:
print("Shape of X",X.shape)
seq_length=X.shape[1]
seq_length

Shape of X (14231, 19)


19

In [29]:

model=Sequential()
model.add(Embedding(vocab_size,50,input_length=seq_length))
#The first layer is the Embedded layer that uses 50-length vectors
#return_sequences=True because we add another LSTM
model.add(LSTM(100,return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100,activation="relu"))
model.add(Dense(vocab_size,activation="softmax"))

In [33]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 19, 50)            69800     
                                                                 
 lstm (LSTM)                 (None, 19, 100)           60400     
                                                                 
 lstm_1 (LSTM)               (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 100)               10100     
                                                                 
 dense_1 (Dense)             (None, 1396)              140996    
                                                                 
Total params: 361696 (1.38 MB)
Trainable params: 361696 (1.38 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [31]:
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])


In [32]:
model.fit(X,y,batch_size=32,epochs=150)

Epoch 1/150
445/445 [==============================] - 20s 34ms/step - loss: 5.6961 - accuracy: 0.0453
Epoch 2/150
445/445 [==============================] - 15s 34ms/step - loss: 5.2601 - accuracy: 0.0500
Epoch 3/150
445/445 [==============================] - 15s 34ms/step - loss: 5.0394 - accuracy: 0.0572
Epoch 4/150
445/445 [==============================] - 16s 36ms/step - loss: 4.8662 - accuracy: 0.0689
Epoch 5/150
445/445 [==============================] - 15s 34ms/step - loss: 4.7220 - accuracy: 0.0782
Epoch 6/150
445/445 [==============================] - 15s 35ms/step - loss: 4.5984 - accuracy: 0.0895
Epoch 7/150
445/445 [==============================] - 16s 37ms/step - loss: 4.4756 - accuracy: 0.1003
Epoch 8/150
445/445 [==============================] - 16s 35ms/step - loss: 4.3411 - accuracy: 0.1187
Epoch 9/150
445/445 [==============================] - 15s 35ms/step - loss: 4.1985 - accuracy: 0.1411
Epoch 10/150
445/445 [==============================] - 16s 36ms/step - l

In [34]:
text_lenght= 15 # 15 words per line

def generate_text(input_text, no_lines):
    general_text=[]
    for i in range(no_lines):
        text=[]
        for _ in range(text_lenght):
            encoded=tokenizer.texts_to_sequences([input_text])
            encoded=pad_sequences(encoded,maxlen=seq_length,padding="pre")
            y_pred=np.argmax(model.predict(encoded),axis=-1) # it will generate a word index, loop up into dictionary containing word index

            predicted_word=""
            for word,index in tokenizer.word_index.items():
                if index==y_pred:
                    predicted_word=word
                    break

            input_text=input_text +' '+ predicted_word
            text.append(predicted_word)

        input_text=text[-1]
        text=" ".join(text) # input text will be the last word of first created line
        general_text.append(text)

    return general_text

In [35]:
input_text="me"
text_produced=generate_text(input_text,6)
text_produced

1/1 [==============================] - 0s 25ms/step


["i'm it make your heart behind me i would know i was so reckless if",
 "you're ready are you ready treat her love and make me imagine you're here find",
 "the things i think just don't make my heart the note that i want you",
 "know when to make me imagine you're here find yourself a forty fool on me",
 "i'm it make your heart behind me i would know i was so reckless if",
 "you're ready are you ready treat her love and make me imagine you're here find"]

In [36]:
input_text="i want to see you"
text_produced=generate_text(input_text,6)
text_produced

1/1 [==============================] - 0s 30ms/step


['never felt close you i remember all of my memories in my heart i been',
 'i i miss my whole life hiding my heart away away woke up feeling heavy',
 'i can know by the way you treat your face under every kind of your',
 "love drives me crazy it's hard to understand just a one for of this would",
 "i have a figure but it's like i'm why this hurts 'cause i m sober",
 'and fell in your mouth the one for you in leave this on words trying']